What is a graph?

What do you want to do on it?


Search a graph
spanning tree
routing on a graph
shortest path
maximum flow
minimum cut
maximum matching
minimum spanning tree
strongly connected components
biconnected components
articulation points
bridges
2-edge connected components


colorings
covers

# Tree Decomposition

https://dl.acm.org/doi/10.1145/3689801  Fast and Optimal Extraction for Sparse Equality Graphs
https://github.com/mabseher/htd hypertree decomposition
https://github.com/PACE-challenge/Treewidth
https://pypi.org/project/treedecomp/
https://networkx.org/documentation/latest/reference/algorithms/approximation.html#module-networkx.algorithms.approximation.treewidth


In [ ]:
# tree decomp
class Node():
    bag: set[V]
    bagedges : set[E]
    children: set[Node]

    def treewidth(self):
        return max(len(self.bag) - 1, max([c.treewidth() for c in self.children]))
    

class GraphTree():
    here: set[V] # nodes only here
    edges: set[E] #egdes between only here and children
    children: set[tuple[set[V], GraphTree]]

    def color(self, colors):
       # take in a list of colors 


In [ ]:
V = int
Edge = tuple[V,V]

# flat reps
Graph1 = set[Edge]
Graph2 = list[Edge]
Graph3 = dict[V, set[V]]


# structured representations

In [ ]:
# kerngihan Lin

def kl(edges):
    A = set()
    B = set()
    



# Graph canonicalization

## group theory
This is neither here nor there kind of. It's "just" for symmettry breaking during search, which is especially painful in canonization search


caonicalizing graphs  via brute force. Come up with a system of serialzing, sort those strings. Not necessarily stable in any sense though
graph isomorphism
Sims-schreiers

schreir vectors = spanning tree. (?) store group element that would connect to previous node of spanning tree. 
This is the group union find? Huh.

schreir sims - a canonical form of group element.

Stabilizer = subgroup defined as the the group elements that leave a particular elemement unchanged

Schreir generators. If we pick representatives of the cosets of a subgroup, we can generate the subgroup using combined generators and repsdentatives as generators.

Return stabilizer of initial point as collection of schreier generatoras. Orbit stabilizer algorithm

undirected Graph where every neighborhood has the same moves = group action on vertex set. edge pattern is generators. 
https://en.wikipedia.org/wiki/Cayley_graph
Origin is special or no?
Stabilizer group of origin are loops.
rubiks cube

https://terrytao.wordpress.com/2010/07/10/cayley-graphs-and-the-geometry-of-groups/
subgroups and bundles

a base, special points.
What if graph is disconnected?

Consider general graphs as mangled cayley graphs.
Even if I was given two cayley graphs is it obvious how to canonicaslize it?
The point of gropup theory is to cut down on boring symmettry explosion. Where is this?

cosets partition... partition refinement using group tags?

g = (h, g')  where h is element of subgroup and g' is caonoical represdentiatvie of coset


In [14]:
from dataclasses import dataclass
@dataclass(frozen=True)
class Perm():
    perm : list[int]
    @classmethod  
    def e(self, n):
        return Perm(list(range(n)))
    def __mul__(self, other):
        return Perm([self.perm[p] for p in other.perm])
    
e3 = Perm.e(3)
assert e3 * e3 == e3
p = Perm([1,0])
assert p * p == Perm.e(2)
assert Perm([1,2,0]) * Perm([2,0,1]) == Perm([0,1,2])


def naive()


In [36]:
import pynauty
g = pynauty.Graph(3, directed=False)
g.connect_vertex(0, [1])
g.connect_vertex(1, [2])
print(g)
print(pynauty.autgrp(g)) #  (generators, grpsize1, grpsize2, orbits, numorbits)
print(pynauty.certificate(g))
print(pynauty.canon_graph(g))


Graph(number_of_vertices=3, directed=False,
 adjacency_dict = {
  0: [1],
  1: [2],
 },
 vertex_coloring = [
 ],
)
([[2, 1, 0]], 2.0, 0, [0, 1, 0], 2)
b'\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\xc0'
Graph(number_of_vertices=3, directed=False,
 adjacency_dict = {
  0: [2],
  1: [2],
  2: [0, 1],
 },
 vertex_coloring = [
 ],
)


## brute force

You can turn a graph of n vertices into a n^2/2 string by listing out all the possible edges
This is the same thing as `{{v1,v2}}` representation.
https://cs.stackexchange.com/questions/14354/simple-graph-canonization-algorithm

https://arxiv.org/abs/1301.1493  Practical graph isomorphism, II

https://cgm.cs.mcgill.ca/~breed/2015COMP362/canonicallabellingpaper.pdf McKay’s Canonical Graph Labeling Algorithm

A key point here is that the representation is really useful.


In [18]:
#G = {{1.2},{2,3},{1,3}}  # can't hash sets. could use frozenset. Anyway
def e(i,j):
    return (i,j) if i < j else (j,i)
def graph(edges):
    return list(sorted(set(edges))) # dedup and sort

G = graph({e(1,2), e(2,3), e(1,3)})


[(1, 2), (1, 3), (2, 3)]

In [20]:
from itertools import permutations
min([ graph([e(p[0], p[1]), e(p[1], p[2])])  for p in permutations(range(3))])

[(0, 1), (0, 2)]

In [21]:
min([ graph([e(p[1], p[0]), e(p[0], p[2])])  for p in permutations(range(3))])

[(0, 1), (0, 2)]

In [ ]:
# a functional rep of graph as a thing permutations can act on.

#G = Callable[perm, list[tuple[int,int]]]
def perm_graph(perm, G): # a permutation acting on a labelled graph
    return [ (p[i], p[j]) for (i,j) in G ]

Suppose I wanted the automorphism group.
Can I generate it?

In [65]:
range(3)[3]

IndexError: range object index out of range

In [61]:
G = sorted([(2,1), (1,0)])
import itertools
def perm_graph(perm, G): # a permutation acting on a labelled graph
    return tuple(sorted([ tuple(sorted([p[i], p[j]])) for (i,j) in G ]))
print(G)
G = perm_graph(range(3), G)
print(G)
# explicitly list the automorphisms of a graph
def canon(G, N):
    autogrp = []
    min_graph = G
    print(G)
    for p in itertools.permutations(range(N)):
        G1 = perm_graph(p, G)
        print(p,G1)
        if G1 == G:
            autogrp.append(p)
        elif G1 < min_graph:
            min_graph = G1
    return autogrp, min_graph
canon(G, 3)

[(1, 0), (2, 1)]
((0, 3), (1, 3))
((0, 3), (1, 3))
(0, 1, 2) ((0, 2), (2, 3))
(0, 2, 1) ((0, 2), (2, 3))
(1, 0, 2) ((0, 2), (2, 3))
(1, 2, 0) ((0, 2), (2, 3))
(2, 0, 1) ((0, 2), (2, 3))
(2, 1, 0) ((0, 2), (2, 3))


([], ((0, 2), (2, 3)))

 swaps
Generate permutation group. swaps

Also, swaps are kind of an incrementalo strcturue which is good for functional programming maybe.

note that this is a method to generate permutations too. swap to decide which element of remaining in in nth position.

https://en.wikipedia.org/wiki/Steinhaus%E2%80%93Johnson%E2%80%93Trotter_algorithm  Steinhaus–Johnson–Trotter algorithm
https://en.wikipedia.org/wiki/Heap%27s_algorithm
https://en.wikipedia.org/wiki/Fisher%E2%80%93Yates_shuffle randomly genrate a permutation

In [ ]:


def canon(G, N):
    swaps = [(i,j) for i in range(N) for j in range(i)]
    for 
    for p in permutations(range(3)):
        yield perm_graph(p, G)

If we do a more careful backtracking search, we can prune a lot.
We learn partial order constraints as we go. Maybe a difference logic thing would be helpful.
We get good strings by ordering the vertices by degree first.
We can do a lot of propagation.
We can do prefix comparison because we're being lexicographic.





In [ ]:
def lt(G1,G2):
    [G1[i,j] and not G2[i,j] for i in len(G1) for j in len(G2)]

# recursive form of lex comparison
# turn if into If. i and j are static.
def lt(G1,G2,i,j):
    if i == len(G1) and j == len(G2):
        return False
    
    if G1[i,j] and not G2[i,j]:
        return True
    elif not G1[i,j] and G2[i,j]:
        return False
    else:
        if i+1 == len(G1):
            return lt(G1,G2,0,j+1)
        else:
            return lt(G1,G2,i+1, j)

# or bottom up loop form
def lt(G1,G2):
    acc = False
    for i in reversed(range(len(G1))):
        for j in reversed(len(G1)):
            if G1[i][j] == G2[i][j]:
                acc = acc
            elif G[i,j] and not G2[i,j]:
                return 
            if not G1[i][j] and G2[i][j]:
                return True
    
def lt(perm,G2,N):
    acc = BoolVal(False)
    for i in reversed(range(N)):
        for j in reversed(range(N)):
            t = G1
            acc = If(
                t[i,j] == G2[i,j],
                acc,
                If(
                    And(Not(t[i,j]), G2[i,j]),
                    BoolVal(True),
                    acc
                )
            )
            if G1(perm(i),perm(j)) == G2(i,j):
                acc = acc
            elif G[i,j] and not G2[i,j]:
                return 
            if not G1[i][j] and G2[i][j]:
                return True

def to_matrix(G):
    mat = [[False]*len(G)] * len(G)
    for (i,j) in G:
        mat[i][j] = True
        mat[j][i] = True
    return mat

We could have other representations of graphs. A tree representation follows naturally from Tree Decomposition (kind of. We also need to create a unique tree decomp... or use an ordering that doesn't care which tree decomp we picked. Hmmmm. best Spanning tree? Again tie breaking here brings in graph canonization. These things need to be considered bolted together).
We could then use total tree orderings instead of string orderings.

"Natrual graph orderings" like that minor homomorphism thing. https://en.wikipedia.org/wiki/Robertson%E2%80%93Seymour_theorem

Maybe this is more sensical for tree with backref ids. Which numbers do we put in the backref ids to make canonical. Then the tree structure exists a priori.


In [ ]:
import z3



def compare_graph(G1,G2):

def canon(G):
    s = z3.Solver()
    Gsym = Function("G", IntSort(), IntSort(), BoolSort())
    Function("perm", IntSort(), IntSort())
    for i in len(G):
        for j in range(i):
            s.add(perm(i) != perm(j)) # must be permutation
    for i,j in G:
        s.add(Gsym(i,j) == True) # "need" to make graph symbolic because perm is symbilic.
    perm = Function("perm", IntSort(), IntSort())
    def lt(perm,G):
        acc = BoolVal(False) # all equal
        for i in range(len(G)):
            for j in range(i):
                acc = If(G[perm[i], perm[j]] < G[i,j], True, acc)
                    
    
    while True:
        m = s.model()
        s.add(compare_graph(G, m))
        if s.check() == unsat:
            return G


Ok, so there is some way of doing that. And maybe this is an example where SAT solvers can really shine. (They have more symmettry breaking in them?)


Clauses as graphs. ordered resolution
Queries as graphs
Tensor Expressions as graphs

Breaking up the permutations via features

Can we use tree decomposition to make a natural graph canonization that elegant moves on from tree like problems. This is like Caleb's thing.
Dynamic programming approach to canonical ordering.

brute force approach to searching all tree decompositions? Exact solvers must be doing this in some sense.

It'd be nice to not a priori fix the ordering. If some particular graph has a particular unique feature, obviously we want to use that.

{foo(A,B), foo(B,C)}
[("foo", p[1], p[2]), ("foo", p[2], p[3])]
We could choose "foo" tags to come later but that'd be crazy (?)

Group union find for labelled graphs being interconnected to each other.


Refinement: order coarsely by colors first.

A pile of ground tensor equations would work.
G --> (P, canon(G))

This is a canonical form of a labelled graph. It's a normalizer. Is this the group union find?



TijKij = Rijkk

tensor instruction selection


true global variables are "observed". They aren't known distinct though.

https://automorphisms.org/
https://arxiv.org/pdf/2406.13557 satsuma Structure-based Symmetry
Breaking in SAT


seress permutation group algorithms

In [22]:
# to get cycles. Orbits under subgroup consisting of p^n. Generated by <p>
def cycles(p):
    found = set()
    for i in range(len(p)):
        if i not in found:
            cycle = []
            j = i
            while j not in cycle:
                cycle.append(j)
                j = p[j]
            yield cycle
            found.update(cycle)

list(cycles([1,0,2]))

def inv(p):
    inv = [0]*len(p)
    for i in range(len(p)):
        inv[p[i]] = i
    return inv

def orbit(ps, n):
    found = set()
    done = False
    while not done:
        done = False
        for f in found:
            for p in ps:
                e = p[f]
                if e not in found:
                    done = False
                    found.add(e)

Perm = list[int]
# orbit of is fixed point of this operation starting from {x}.
def step(G : list[Perm], s : set[int]) -> set[int]: # G is list/set of permutations
    return s | {p[i] for p in G for i in s}

def is_inv(G,s):
    return s == step(G,s)

# can apply standard seminaive trick
# can retain "proof" / path as group union find.

# schreier vectors


# base
# strong generating set



[[0, 1], [2]]

# tensor canon
https://docs.sympy.org/latest/modules/combinatorics/tensor_can.html

https://docs.sympy.org/latest/modules/tensor/tensor.html#sympy.tensor.tensor.TensAdd.canon_bp

In [17]:
from sympy.tensor.tensor import TensorIndexType, tensor_indices, TensorHead, TensorManager, TensorSymmetry
IType = TensorIndexType('IType')
i,j,k,l = tensor_indices('i j k l', IType)
A = TensorHead('A', [IType, IType])
A
G = TensorHead('G', [IType], TensorSymmetry.no_symmetry(1), 'Gcomm')
G(j)
t = A(i,j)*A(-j,k)*A(-k,l)
t2 = A(-k,j)*A(i,k)*A(-j,l)
t.canon_bp() == t2.canon_bp()

True

In [27]:
import itertools
i,j,k,l = range(4)
t = [("A", i,k), ("A", j,l), ("A", k,l)]

def act(p, t):
    return sorted([(head,*[p[a] for a in args]) for head,*args in t])

def normal(t, N):
    return min([ (p, act(p,t)) for p in itertools.permutations(range(N))], key=lambda x: x[1])

assert act([0,1,2,3], t) == t

p,t1 = normal(t, 4)
assert act(p,t) == t1

A symmettric tensor has a multiset of indices. This is a graph? multigraph? hypergraph?
```
def act(p, t):
    return sorted([(head,*sorted([p[a] for a in args])) for head,*args in t])
```


foo(g(i), k(j), l(i)) -> ("foo", ("g", i), ("k", j), ("l", i)). Permutation action + min. Brute force search is silly because we will just get the dpeth first relabelling ordering.

But this is vistiation lex ordering.

What about KBO, LPO etc
KBO - they will all be the same size. Unless we weight the indice choice. This could make sense. We want fewer repeated indices. They represent coupling.
Order by number of appareances. Tie break by recursion. https://www.philipzucker.com/ground_kbo/ This is not substitution stable.
Can be lazy about pushing permutations down once determined.




In [ ]:
def termact(p, t):
    head,*args = t
    return (head,*[termact(p,a) for a in args])
def canon(t, N):
    # a silly implementation.
    min([ termact(p,t) for p in itertools.permutations(range(N))])



## cadbra
https://cadabra.science/

https://docs.sympy.org/latest/modules/combinatorics/tensor_can.html
"
The Butler-Portugal algorithm [3] is an efficient algorithm to put tensors in canonical form, solving the above problem.

Portugal observed that a tensor can be represented by a permutation, and that the class of tensors equivalent to it under slot and dummy symmetries is equivalent to the double coset 
 (Note: in this documentation we use the conventions for multiplication of permutations p, q with (p*q)(i) = p[q[i]] which is opposite to the one used in the Permutation class)
 "
https://arxiv.org/abs/1702.08114 Faster Tensor Canonicalization
https://www.sciencedirect.com/science/article/abs/pii/S001046550800221X xPerm: fast index canonicalization for tensor computer algebra

https://pure.mpg.de/rest/items/item_1833414/component/file_2047302/content  xTras: A field-theory inspired xAct package for mathematica
https://europepmc.org/article/pmc/pmc6105178  Computer algebra in gravity research. This paper rules
redberry is a tensor CAS?
https://durham-repository.worktribe.com/OutputFile/1927714 Hiding canonicalisation in tensor computer algebra
https://etheses.dur.ac.uk/14811/1/thesis.pdf?DDD21+  thesis version
cadabra peeters

https://www.frontiersin.org/journals/astronomy-and-space-sciences/articles/10.3389/fspas.2020.00058/full intropductin to numerical relativity
https://github.com/wojciechczaja/GraviPy Tensor Calculus Package for General Relativity based on SymPy (python library for symbolic mathematics).

`sudo apt install cadabra2`

young projectors

HJe uses latex `$` asa quote for metaprogramming. Interesting.

In [46]:
#https://cadabra.science/static/cadabra_in_ipython.nb.html
from cadabra2 import *
from IPython.display import display, Math, Latex
ex=Ex(r"A_{m n} B^{m n}") # latex expressions
display(Math(str(ex)))

# assign properties to symbols
Symmetric(Ex(r"A_{m n}"))
AntiSymmetric(Ex(r"B_{m n"))

display(Math(str(canonicalise(ex))))

Indices(Ex(r"{m,n,p,q,r,s,t}"))
ex=Ex(r"A_{m n} B^{m p} C_{p q}")
display(Math(str(substitute(ex, Ex(r"A_{m n} -> D_{m q} D^{q}_{n}")))))
display(Math(str(substitute(ex, Ex(r"A_{k n} -> D_{k q} D^{q}_{n}"))))) # so it looks like it is matching modulo names

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

https://josmar493.dreamhosters.com/ xACT. mathemtica analog?
https://www.youtube.com/watch?v=qOaJxzezU8w&list=PLdIcYTEZ4S8TSEk7YmJMvyECtF-KA1SQ2&ab_channel=WolframR%26D web series 20 year of 